In [1]:
import pandas as pd
import json

# --- 1. Load your JSON ---
# Replace 'your_file.json' with the path to your JSON file
with open('data/LesOutputJson.json', encoding='utf-8') as f:
    json_data = json.load(f)

data = json_data['Data']

In [2]:
# --- 2. Extract main company info ---
main_cols = [
    "Регистрийн дугаар",
    "Оноосон нэр",
    "Бүртгэсэн огноо",
    "Хэлбэр",
    "Төрөл"
]

df_company = pd.DataFrame([row[:5] for row in data], columns=main_cols)

# --- Helper function to extract nested arrays ---
def extract_nested(data, index, columns):
    rows = []
    for row in data:
        reg = row[0]  # Регистрийн дугаар
        nested_list = row[index] if row[index] else []
        for item in nested_list:
            record = {"Регистрийн дугаар": reg}
            record.update({col: item[i] for i, col in enumerate(columns)})
            rows.append(record)
    return pd.DataFrame(rows)

# --- 3. Shareholders ---
shareholder_cols = ["Ангилал", "Улсын нэр", "Эцэг/эх/-ийн нэр", "Нэр", "Хүйс", "Бүртгэсэн огноо"]
df_shareholders = extract_nested(data, 5, shareholder_cols)

# --- 4. Representatives ---
rep_cols = ["Албан тушаал", "Улсын нэр", "Эцэг/эх/-ийн нэр", "Нэр", "Хүйс", "Бүртгэсэн огноо"]
df_representatives = extract_nested(data, 6, rep_cols)

# --- 5. Final owners ---
owner_cols = ["Ангилал", "Эцэг/эх/-ийн нэр", "Нэр", "Хүйс", "Бүртгэсэн огноо"]
df_final_owners = extract_nested(data, 7, owner_cols)

# --- 6. Business activities ---
activity_cols = ["Төлөв", "Бүртгэсэн огноо", "Үйл ажиллагааны чиглэл"]
df_activities = extract_nested(data, 8, activity_cols)

# --- 7. Reorganization history ---
reorg_cols = ["Өөрчлөгдөн зохион байгуулсан хэлбэр", "Бүртгэсэн огноо", "Өөрчлөлтийн өмнөх оноосон нэр", "Өөрчлөлтийн тэмдэглэл"]
df_reorg = extract_nested(data, 9, reorg_cols)

# --- 8. Quick check ---
print("=== Company Table ===")
print(df_company.head())
print("\n=== Shareholders ===")
print(df_shareholders.head())
print("\n=== Representatives ===")
print(df_representatives.head())
print("\n=== Final Owners ===")
print(df_final_owners.head())
print("\n=== Activities ===")
print(df_activities.head())
print("\n=== Reorganizations ===")
print(df_reorg.head())

# ✅ All DataFrames ready for analysis


=== Company Table ===
  Регистрийн дугаар           Оноосон нэр Бүртгэсэн огноо  \
0           3199665       Ташир агро фийд      2021.03.05   
1           6673252                  РБМК      2021.03.05   
2           6678912     Уудам баян бөөрөг      2021.03.16   
3           6678971  Хан технологийн гүүр      2021.03.16   
4           6678904     Нүүдэлчин их оньс      2021.03.16   

                                 Хэлбэр             Төрөл  
0  Хязгаарлагдмал хариуцлагатай компани      Ашгийн төлөө  
1  Хязгаарлагдмал хариуцлагатай компани      Ашгийн төлөө  
2  Хязгаарлагдмал хариуцлагатай компани      Ашгийн төлөө  
3  Хязгаарлагдмал хариуцлагатай компани      Ашгийн төлөө  
4                        Холбоо /НҮТББ/  Ашгийн төлөө бус  

=== Shareholders ===
  Регистрийн дугаар Ангилал   Улсын нэр Эцэг/эх/-ийн нэр         Нэр Хүйс  \
0           3199665   Иргэн  Монгол Улс       даваасүрэн  эрдэнэтуяа   Эм   
1           6673252   Иргэн  Монгол Улс      цэдэнбалжир  идэрбүргэд   Эр  

In [3]:
df_company['Регистрийн дугаар'].value_counts()

Регистрийн дугаар
7083889    2
6807941    2
4419154    2
4112873    2
6808263    2
          ..
1044109    1
8018154    1
1042157    1
3076016    1
3449181    1
Name: count, Length: 275295, dtype: int64

In [4]:
df_company.shape

(275342, 5)

In [5]:
df_company = df_company.drop_duplicates()
df_company.shape

(275295, 5)

# Data cleaning

Company main info table -> Raw: 275,342 rows -> Drop duplicates -> 275,295 rows
Shareholders info table -> Raw: 633,515 rows -> Drop duplicates -> 623,398 rows
Representitives info table -> Raw: 245,917 rows -> Drop duplicates -> 245,872 rows
Final owners info table -> Raw: 326,128 -> Drop duplicates -> 326,041 rows
Activities info table -> Raw: 1,013,536 -> Drop duplicates -> 1,011,030 rows
Company reorganization info table -> Raw: 543 rows -> Drop duplicates -> 539 rows

In [6]:
df_shareholders.shape

(633515, 7)

In [7]:
df_shareholders

,Регистрийн дугаар,Ангилал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо
0,3199665,Иргэн,Монгол Улс,даваасүрэн,эрдэнэтуяа,Эм,2021.03.05
1,6673252,Иргэн,Монгол Улс,цэдэнбалжир,идэрбүргэд,Эр,2021.03.05
2,6678912,Иргэн,Монгол Улс,оюундэлгэр,амарсанаа,Эр,2021.03.16
3,6678971,Иргэн,Монгол Улс,дамбадаржаа,алимаа,Эм,2023.04.14
4,6678939,Иргэн,Монгол Улс,уртнасан,болорчимэг,Эм,2024.05.22
...,...,...,...,...,...,...,...
633510,3783537,Иргэн,Монгол Улс,эрхэмбаяр,мөнгөншагай,Эр,2025.08.13
633511,7182785,Иргэн,Монгол Улс,батбилэг,түшиг,Эр,2025.08.13
633512,3404757,Х/Этгээд,Монгол Улс,,Бэхгүрэн,,2025.08.13
633513,7182985,Иргэн,Монгол Улс,ганболд,мягмардорж,Эр,2025.08.13


In [8]:
df_representatives

,Регистрийн дугаар,Албан тушаал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо
0,3199665,Гүйцэтгэх захирал,Монгол Улс,Даваасүрэн,Эрдэнэтуяа,Эм,2021.03.05
1,6673252,Гүйцэтгэх захирал,Монгол Улс,Цэдэнбалжир,Идэрбүргэд,Эр,2021.03.05
2,6678912,Гүйцэтгэх захирал,Монгол Улс,Оюундэлгэр,Амарсанаа,Эр,2021.03.16
3,6678971,Гүйцэтгэх захирал,Монгол Улс,Дамбадаржаа,Алимаа,Эм,2023.04.14
4,6678904,Гүйцэтгэх захирал,Монгол Улс,Нэргүй,Балдандорж,Эр,2021.03.16
...,...,...,...,...,...,...,...
245912,7182785,Гүйцэтгэх захирал,Монгол Улс,Батбилэг,Түшиг,Эр,2025.08.13
245913,3404757,Гүйцэтгэх захирал,Монгол Улс,Батзоригт,Батбаяр,Эр,2025.08.13
245914,7182985,Гүйцэтгэх захирал,Монгол Улс,Ганболд,Мягмардорж,Эр,2025.08.13
245915,3267989,Гүйцэтгэх захирал,Монгол Улс,Ойдов,Батжаргал,Эм,2025.08.14


In [9]:
df_representatives.shape

(245917, 7)

In [10]:
df_representatives = df_representatives.drop_duplicates()
df_representatives.shape

(245872, 7)

In [11]:
df_final_owners

,Регистрийн дугаар,Ангилал,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо
0,6673252,Иргэн,Идэрбүргэд,Цэдэнбалжир,Монгол Улс,Эр
1,6678912,Иргэн,Амарсанаа,Оюундэлгэр,Монгол Улс,Эр
2,6678971,Иргэн,Алимаа,Дамбадаржаа,Монгол Улс,Эм
3,6678904,Иргэн,Балдандорж,Нэргүй,Монгол Улс,Эр
4,6678939,Иргэн,Болорчимэг,Уртнасан,Монгол Улс,Эм
...,...,...,...,...,...,...
326123,7182785,Иргэн,Түшиг,Батбилэг,Монгол Улс,Эр
326124,3404757,Иргэн,Батбаяр,Батзоригт,Монгол Улс,Эр
326125,7182985,Иргэн,Мягмардорж,Ганболд,Монгол Улс,Эр
326126,3267989,Иргэн,Сарангэрэл,Няндагцэрэн,Монгол Улс,Эм


In [12]:
df_final_owners.shape

(326128, 6)

In [13]:
df_final_owners = df_final_owners.drop_duplicates()
df_final_owners.shape

(326041, 6)

In [14]:
df_activities

,Регистрийн дугаар,Төлөв,Бүртгэсэн огноо,Үйл ажиллагааны чиглэл
0,3199665,Тийм,2021.03.05,"Газар тариалан /Үр тариа, төмс хүнсний ногоо, ..."
1,3199665,Тийм,2021.03.05,Мал аж ахуй
2,6673252,Тийм,2021.03.05,"Төмөр хашаа, хайс, хаалт хийх"
3,6673252,Тийм,2021.03.05,Хэвлэлийн эх бэлтгэх
4,6673252,Тийм,2021.03.05,Барилгын материалын худалдаа
...,...,...,...,...
1013531,7182985,Тийм,2025.08.13,Аялал жуулчлал
1013532,3267989,Тийм,2025.08.14,"Хүүхэд гэр бүл, албан байгууллага сэтгэлзүйн з..."
1013533,3449181,Тийм,2025.08.14,Төлбөр эсвэл гэрээний үндсэн дээр хийгдэх бөөн...
1013534,3449181,Тийм,2025.08.14,Хөгжмийн зэмсэг үйлдвэрлэл


In [15]:
df_activities.shape

(1013536, 4)

In [16]:
df_activities = df_activities.drop_duplicates()
df_activities.shape

(1011030, 4)

In [17]:
df_reorg

,Регистрийн дугаар,Өөрчлөгдөн зохион байгуулсан хэлбэр,Бүртгэсэн огноо,Өөрчлөлтийн өмнөх оноосон нэр,Өөрчлөлтийн тэмдэглэл
0,5841127,Нэгтгэх,11-APR-16,САТАС корпораци,5384621 тоот регистрийн дугаартай Сатапартнерс...
1,5841127,Нэгтгэх,12-MAY-17,САТАС корпораци,5440866 тоот регистрийн дугаартай Сатакашемир ...
2,5841127,Нэгтгэх,20-JAN-20,САТАС корпораци,5669928 тоот регистрийн дугаартай Оргилгурван ...
3,5414105,Нэгтгэх,24-DEC-20,Нэмэхдэвшил,3859436 тоот регистрийн дугаартай Жэй Эй Икс-г...
4,5414105,Нэгтгэх,24-MAR-21,Нэмэхдэвшил,3859436 тоот регистрийн дугаартай Жэй Эй Икс-г...
...,...,...,...,...,...
538,6577539,Нэгтгэх,30-DEC-21,Иргэний нисэхийн үндэсний төв,5055954 тоот регистрийн дугаартай Буянт-Ухаа о...
539,6577539,Нэгтгэх,30-DEC-21,Иргэний нисэхийн үндэсний төв,5055954 тоот регистрийн дугаартай Буянт-Ухаа о...
540,3897907,Шинэ бүртгэл,03-SEP-25,Төв аймгийн Зуунмод сумын ерөнхий боловсролын ...,
541,4040694,Нийлүүлэх,27-NOV-25,Баруун бүсийн цахилгаан түгээх сүлжээ,2684853 тоот регистрийн дугаартай Ховдцахилгаа...


In [18]:
df_reorg = df_reorg.drop_duplicates()
df_reorg.shape

(539, 5)

In [61]:
df_shareholders[(df_shareholders['Эцэг/эх/-ийн нэр']=='ганбат')&(df_shareholders['Нэр']=='чинзоригт')]

,Регистрийн дугаар,Ангилал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо


In [64]:
df_shareholders[df_shareholders['Регистрийн дугаар']==2693321]

,Регистрийн дугаар,Ангилал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо


In [68]:
df_company[df_company['Регистрийн дугаар']==2693321]

,Регистрийн дугаар,Оноосон нэр,Бүртгэсэн огноо,Хэлбэр,Төрөл


In [67]:
df_company

,Регистрийн дугаар,Оноосон нэр,Бүртгэсэн огноо,Хэлбэр,Төрөл
0,3199665,Ташир агро фийд,2021.03.05,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
1,6673252,РБМК,2021.03.05,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
2,6678912,Уудам баян бөөрөг,2021.03.16,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
3,6678971,Хан технологийн гүүр,2021.03.16,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
4,6678904,Нүүдэлчин их оньс,2021.03.16,Холбоо /НҮТББ/,Ашгийн төлөө бус
...,...,...,...,...,...
275337,7182785,Эгдүүтэй саарал бүжин,2025.08.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
275338,3404757,Степ гласс,2025.08.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
275339,7182985,Бадрах соёмбо,2025.08.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
275340,3267989,Аюурлан,2025.08.14,Холбоо /НҮТББ/,Ашгийн төлөө бус


In [72]:
df_company['Бүртгэсэн огноо'].min()

''

In [76]:
df_company = df_company[df_company['Бүртгэсэн огноо']!='']

In [77]:
df_company

,Регистрийн дугаар,Оноосон нэр,Бүртгэсэн огноо,Хэлбэр,Төрөл
0,3199665,Ташир агро фийд,2021.03.05,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
1,6673252,РБМК,2021.03.05,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
2,6678912,Уудам баян бөөрөг,2021.03.16,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
3,6678971,Хан технологийн гүүр,2021.03.16,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
4,6678904,Нүүдэлчин их оньс,2021.03.16,Холбоо /НҮТББ/,Ашгийн төлөө бус
...,...,...,...,...,...
275337,7182785,Эгдүүтэй саарал бүжин,2025.08.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
275338,3404757,Степ гласс,2025.08.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
275339,7182985,Бадрах соёмбо,2025.08.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
275340,3267989,Аюурлан,2025.08.14,Холбоо /НҮТББ/,Ашгийн төлөө бус


In [78]:
df_company['Бүртгэсэн огноо'].min()

'1926.08.20'

In [79]:
df_company['Бүртгэсэн огноо'].max()

'2025.12.15'

In [81]:
df_company.sort_values('Бүртгэсэн огноо')[:20]

,Регистрийн дугаар,Оноосон нэр,Бүртгэсэн огноо,Хэлбэр,Төрөл
136385,9029362,Дорнод аймгийн тагнуулын газар,1926.08.20,"Төрийн байгууллага, албан газар",Ашгийн төлөө бус
179971,1800094,Олон улсын агаарын тээврийн Эйр Чайна,1991.12.19,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
152235,1800108,Сумитомо корпораци,1991.12.25,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
149092,1800582,Марубени корпораци,1994.02.24,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
92411,2159759,Уурхайт,1996.02.07,Нөхөрлөл /ЗБН/,Ашгийн төлөө
148566,1801333,Эсүандоэсү ХК,1996.10.04,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
154384,1801589,Европ Трейдинг БВ,1997.04.18,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
240380,1801562,Рэдбурне Гоулд компани,1997.04.18,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
155329,1801651,Аэрофлот-Оросын олон улсын агаарын зам хувьцаа...,1997.05.17,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус
150152,1802534,Мицубиши корпораци,2000.02.16,Гадаадын хуулийн этгээдийн төлөөлөгчийн газар,Ашгийн төлөө бус


In [88]:
df_company[df_company['Бүртгэсэн огноо']=='2005.11.04'].sort_values('Оноосон нэр', ascending=False)[:30]

,Регистрийн дугаар,Оноосон нэр,Бүртгэсэн огноо,Хэлбэр,Төрөл
71375,2725533,Өсөхлуу,2005.11.04,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
72041,4368002,Өрлөгзэлмэ,2005.11.04,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
97869,2564394,Эххөгжил,2005.11.04,Гадаадын хөрөнгө оруулалттай хязгаарлагдмал ха...,Ашгийн төлөө
93464,5027462,Энх-Асрал,2005.11.04,Гадаадын хөрөнгө оруулалттай хязгаарлагдмал ха...,Ашгийн төлөө
94918,5027616,Эксэфийл,2005.11.04,Гадаадын хөрөнгө оруулалттай хязгаарлагдмал ха...,Ашгийн төлөө
76167,2622416,Цэнгүүндалай,2005.11.04,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
7957,5027675,Холистик,2005.11.04,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
158218,5027667,Хасхүлэг,2005.11.04,Гадаадын хөрөнгө оруулалттай хязгаарлагдмал ха...,Ашгийн төлөө
128584,2693321,ХААН банк,2005.11.04,Хувьцаат компани /нээлттэй ХК/,Ашгийн төлөө
10295,5027713,Фрэндшип наран,2005.11.04,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө


In [90]:
df_company[df_company['Регистрийн дугаар']=='2693321']

,Регистрийн дугаар,Оноосон нэр,Бүртгэсэн огноо,Хэлбэр,Төрөл
128584,2693321,ХААН банк,2005.11.04,Хувьцаат компани /нээлттэй ХК/,Ашгийн төлөө


In [92]:
df_shareholders[df_shareholders['Регистрийн дугаар']=='2693321']

,Регистрийн дугаар,Ангилал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо
288958,2693321,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.09.06
288959,2693321,Иргэн,Монгол Улс,дияр,еркебек,Эр,2022.09.06
288960,2693321,Иргэн,Монгол Улс,эрдэнэбат,ууганчимэг,Эм,2022.09.06
288961,2693321,Иргэн,Монгол Улс,адъяадорж,эрдэнэмөнх,Эр,2022.09.06
288962,2693321,Иргэн,Монгол Улс,даринчулуун,долгорсүрэн,Эм,2022.09.06
...,...,...,...,...,...,...,...
293095,2693321,Иргэн,Монгол Улс,юндэндорж,энхзул,Эм,2022.09.06
293096,2693321,Иргэн,Монгол Улс,энхболд,батэрдэнэ,Эр,2022.09.06
293097,2693321,Иргэн,Монгол Улс,баясгалан,билгүүн,Эр,2022.09.06
293098,2693321,Иргэн,Монгол Улс,алтантуяа,ганчимэг,Эм,2022.09.06


In [97]:
df_filter = df_shareholders[(df_shareholders['Эцэг/эх/-ийн нэр']=='дашдаваа')&(df_shareholders['Нэр']=='хулан')]
df_filter

,Регистрийн дугаар,Ангилал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо
2168,6643744,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2020.12.16
12411,2543303,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.11.28
170873,5803403,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.11.25
180446,2160544,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2024.02.02
288958,2693321,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.09.06
415022,5646804,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2018.01.16
521121,7002913,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2024.03.15


In [100]:
pd.merge(df_filter, df_company, on="Регистрийн дугаар", how="inner")

,Регистрийн дугаар,Ангилал,Улсын нэр,Эцэг/эх/-ийн нэр,Нэр,Хүйс,Бүртгэсэн огноо_x,Оноосон нэр,Бүртгэсэн огноо_y,Хэлбэр,Төрөл
0,6643744,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2020.12.16,Ар И Ди эженси,2020.12.16,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
1,2543303,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.11.28,Таван богд холдингс,2006.01.13,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
2,5803403,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.11.25,Таван богд фүүдс пицца,2014.02.20,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
3,2160544,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2024.02.02,Буянт-Өрх,2006.05.24,Нөхөрлөл /ЗБН/,Ашгийн төлөө
4,2693321,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2022.09.06,ХААН банк,2005.11.04,Хувьцаат компани /нээлттэй ХК/,Ашгийн төлөө
5,5646804,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2018.01.16,Таванбогд фүүдс,2012.11.01,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө
6,7002913,Иргэн,Монгол Улс,дашдаваа,хулан,Эм,2024.03.15,Вайт маркетинг эйженси,2024.03.15,Хязгаарлагдмал хариуцлагатай компани,Ашгийн төлөө


In [19]:
df_company.to_csv("data/company_main_info.csv", encoding='utf-8-sig',index=False)

In [20]:
df_shareholders.to_csv("data/shareholders_info.csv", encoding='utf-8-sig',index=False)

In [21]:
df_representatives.to_csv("data/reprentatives.csv", encoding='utf-8-sig',index=False)

In [22]:
df_final_owners.to_csv("data/beneficials.csv", encoding='utf-8-sig',index=False)